In [1]:
import torch
from torch import nn

In [2]:
batch = 1
seq = 3
input_size = 3
hidden_size = 2
D = 1
num_layers = 1

In [3]:
gru = nn.GRU(input_size=input_size,hidden_size=hidden_size,bias=False,
             num_layers=num_layers,batch_first=True)
gru

GRU(3, 2, bias=False, batch_first=True)

In [4]:
gru._all_weights

[['weight_ih_l0', 'weight_hh_l0']]

* ih --> input to hidden
* hh --> hidden to hidden
* l0 --> 0th layer

Since GRU have three gates , 
1. Reset Gate, Update Gate, New Gate
1. each gate have its own weight matrix for both input and hidden.
2. each gate matrix are in shape [hidden,hidden or input]
3. three gate matrix for hidden so , W_hh have shape [3*hidden , hidden]
4. three gate matrix for input , W_ih have shape [3*hidden , input]

In [5]:
gru.weight_ih_l0.shape
#? 3*hidden , input

torch.Size([6, 3])

In [6]:
gru.weight_hh_l0.shape
#? 3*hidden , hidden

torch.Size([6, 2])

In [7]:
sample_input = torch.rand(batch,seq,input_size)
sample_input

tensor([[[0.2642, 0.4186, 0.8494],
         [0.8219, 0.1427, 0.9215],
         [0.3571, 0.4470, 0.1099]]])

In [8]:
h = torch.rand(D*num_layers,batch,hidden_size)
h

tensor([[[0.9453, 0.4464]]])

In [9]:
W_ih,W_hh = gru._flat_weights

In [10]:
W_ir,W_iz,W_in = W_ih.split(hidden_size)
W_ir.shape,W_iz.shape,W_in.shape

(torch.Size([2, 3]), torch.Size([2, 3]), torch.Size([2, 3]))

In [11]:
W_hr,W_hz,W_hn = W_hh.split(hidden_size)
W_hr.shape,W_hz.shape,W_hn.shape

(torch.Size([2, 2]), torch.Size([2, 2]), torch.Size([2, 2]))

In [12]:
gru(sample_input,h)

(tensor([[[ 0.2511,  0.5618],
          [-0.2102,  0.6566],
          [-0.1283,  0.5239]]], grad_fn=<TransposeBackward1>),
 tensor([[[-0.1283,  0.5239]]], grad_fn=<StackBackward0>))

In [13]:
output = []
sample_input = sample_input.permute(1,0,2)
for t in range(seq):
    xt = sample_input[t]
    rt = torch.sigmoid(xt@W_ir.T + h@W_hr.T)
    zt = torch.sigmoid(xt@W_iz.T + h@W_hz.T)
    nt = torch.tanh(xt@W_in.T + rt*(h@W_hn.T))
    h = (1-zt)*nt + zt*h
    output.append(h)

In [14]:
torch.stack(output)

tensor([[[[ 0.2511,  0.5618]]],


        [[[-0.2102,  0.6566]]],


        [[[-0.1283,  0.5239]]]], grad_fn=<StackBackward0>)

In [15]:
h

tensor([[[-0.1283,  0.5239]]], grad_fn=<AddBackward0>)